# AI Multi-Agent E-Commerce Fraud & Customer Support Orchestrator

This notebook implements an **enterprise-style multi-agent system** for e-commerce support that can:

- Understand what the customer wants (intent classification)  
- Detect fraud risk in the message and metadata  
- Analyse customer emotion / tone (sentiment)  
- Generate polite, context-aware replies  
- Decide whether to **escalate to a human agent** or not  
- Maintain a small **conversation memory** for context

All of this is done using **pure Python (no external APIs)** so the notebook is fully reproducible and easy to run anywhere.


In [19]:

# =========================
# Cell 1 — Imports & Setup
# =========================

import json
import re
from datetime import datetime
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional


In [20]:
# ====================
# Cell 2 — Memory System
# ====================

@dataclass
class Memory:
    """
    Conversation memory:
    - stores recent messages with role (user/assistant), content, metadata & time
    - useful for showing short context window
    """
    messages: List[Dict[str, Any]] = field(default_factory=list)
    max_history: int = 50

    def add(self, role: str, content: str, meta: Optional[Dict[str, Any]] = None):
        self.messages.append(
            {
                "role": role,
                "content": content,
                "meta": meta or {},
                "time": datetime.now().isoformat(timespec="seconds"),
            }
        )
        if len(self.messages) > self.max_history:
            self.messages = self.messages[-self.max_history :]

    def get_recent_context(self, n: int = 5) -> str:
        """Small text context string from last n messages."""
        out_lines = []
        for m in self.messages[-n:]:
            who = m["role"].upper()
            out_lines.append(f"{who}: {m['content']}")
        return "\n".join(out_lines)

    def to_list(self, n: int = 10) -> List[Dict[str, Any]]:
        """Return last n messages as list (for JSON output)."""
        return self.messages[-n:]


## System Architecture

The system is built as **5 lightweight agents** controlled by a **central Coordinator**:

1. **Intent Agent**
   - Classifies the user's message into high-level intents such as:
     - `refund`, `cancellation`, `billing`, `account_issue`, `delivery_issue`, `fraud_report`, `feedback`, `general_help`
   - Also assigns an **urgency level**: `low`, `medium`, `high`, `critical`.

2. **Sentiment Agent**
   - Rule-based text sentiment:
     - Labels like: `very_angry`, `frustrated`, `neutral`, `positive`, `very_positive`
   - Counts positive and negative words and computes a sentiment score.

3. **Fraud Detection Agent**
   - Estimates **fraud risk** based on:
     - Message content (keywords such as “unknown charge”, “not me”, “scam”, “hacked”)
     - Optional metadata (account age, prior chargeback, IP mismatch, high order value, etc.)
   - Outputs:
     - `risk_level`: `low` / `medium` / `high`
     - `risk_score`: 0.0 – 1.0
     - `reasons`: human-readable explanations.

4. **Reply Agent**
   - Generates a **natural language reply** based on:
     - Intent  
     - Urgency  
     - Sentiment label  
     - Fraud risk level  
   - Adjusts tone if the user is angry/frustrated or positive.

5. **Escalation Agent**
   - Decides whether the issue should be **escalated to a human**.
   - Uses three signals:
     - urgency (from Intent Agent)
     - sentiment (from Sentiment Agent)
     - fraud risk (from Fraud Agent)
   - Returns:
     - `escalate`: `True` / `False`
     - `queue`: `normal`, `priority_support`, or `fraud_team`
     - `reasons`: small explanation list.

6. **Memory**
   - Simple in-notebook memory of recent messages.
   - Stores:
     - role (`user` / `assistant`)
     - content
     - optional metadata
     - timestamp
   - Used to provide a **short context window** for the coordinator.

7. **Coordinator**
   - Orchestrates the full pipeline:
     1. Stores user message in memory
     2. Runs Intent, Sentiment and Fraud agents
     3. Generates a reply
     4. Runs Escalation logic
     5. Stores assistant reply in memory
     6. Returns a combined structured JSON-like output


In [21]:
# ============================
# Cell 3 — Intent Classifier Agent
# ============================

class IntentAgent:
    """
    Classifies a user's message into:
    - intent  : type of request
    - urgency : low / medium / high / critical
    """

    def classify(self, message: str) -> Dict[str, str]:
        text = message.lower()

        intent = "general_help"
        urgency = "low"

        if any(k in text for k in ["refund", "money back", "return my money"]):
            intent, urgency = "refund", "high"

        elif any(k in text for k in ["cancel", "unsubscribe", "stop my subscription"]):
            intent, urgency = "cancellation", "high"

        elif any(
            k in text
            for k in [
                "invoice",
                "billing",
                "billed",
                "charged twice",
                "double charge",
                "payment issue",
                "overcharged",
            ]
        ):
            intent, urgency = "billing", "medium"

        elif any(
            k in text
            for k in [
                "login",
                "log in",
                "password",
                "otp",
                "verification code",
                "can't sign in",
                "cannot sign in",
            ]
        ):
            intent, urgency = "account_issue", "high"

        elif any(
            k in text
            for k in [
                "delay",
                "late delivery",
                "delivery",
                "shipment",
                "shipping",
                "not delivered",
                "still waiting",
            ]
        ):
            intent, urgency = "delivery_issue", "medium"

        elif any(
            k in text
            for k in [
                "fraud",
                "scam",
                "phishing",
                "not me",
                "i didn't order",
                "did not order",
                "unknown charge",
                "card charged",
                "hacked",
            ]
        ):
            intent, urgency = "fraud_report", "critical"

        elif any(k in text for k in ["thank you", "thanks", "awesome", "great service", "love it"]):
            intent, urgency = "feedback", "low"

        elif "help" in text or "support" in text:
            intent, urgency = "general_help", "medium"

        return {"intent": intent, "urgency": urgency}


## End-to-End Flow

The overall flow for a single user message looks like this:

```text
User Message
     |
     v
Intent Agent ─────► intent + urgency
     |
     v
Sentiment Agent ─► sentiment label + score
     |
     v
Fraud Agent ─────► fraud risk level + reasons
     |
     v
Reply Agent ─────► final natural-language reply
     |
     v
Escalation Agent ► escalate? which queue?
     |
     v
Coordinator ─────► combined JSON result + memory update


In [22]:
# ===============================
# Cell 4 — Sentiment Analysis Agent
# ===============================

class SentimentAgent:
    """
    Lightweight rule-based sentiment:
    - label: very_angry / frustrated / neutral / positive / very_positive
    - score: positive_count - negative_count
    """

    POSITIVE_WORDS = {
        "great",
        "good",
        "awesome",
        "amazing",
        "love",
        "nice",
        "thanks",
        "thank",
        "happy",
        "fast",
        "quick",
        "excellent",
    }

    NEGATIVE_WORDS = {
        "terrible",
        "worst",
        "bad",
        "angry",
        "upset",
        "disgusting",
        "hate",
        "useless",
        "cheated",
        "scam",
        "fraud",
        "unacceptable",
        "delay",
        "late",
        "horrible",
    }

    def analyze(self, message: str) -> Dict[str, Any]:
        text = message.lower()
        tokens = re.findall(r"\w+", text)

        pos = sum(1 for t in tokens if t in self.POSITIVE_WORDS)
        neg = sum(1 for t in tokens if t in self.NEGATIVE_WORDS)
        score = pos - neg

        if score <= -3:
            label = "very_angry"
        elif score in (-2, -1):
            label = "frustrated"
        elif score == 0:
            label = "neutral"
        elif score in (1, 2):
            label = "positive"
        else:
            label = "very_positive"

        return {
            "label": label,
            "score": score,
            "positive_matches": pos,
            "negative_matches": neg,
        }


Real-World Use-Case

This system is designed around a **real e-commerce scenario** where:

- Customers report issues like:
  - delayed deliveries  
  - double charges  
  - unknown payments  
  - account problems  
  - refund and cancellation requests
- Companies need to:
  - detect potential fraud early  
  - understand how frustrated the customer is  
  - prioritise and escalate the right tickets  
  - send fast and consistent replies  

By combining **multi-agent orchestration, fraud heuristics, sentiment analysis and escalation logic**, this notebook behaves much closer to how **actual enterprise support tools** work behind the scenes.


In [23]:
# ============================
# Cell 5 — Fraud Detection Agent
# ============================

class FraudAgent:
    """
    Estimates fraud risk using:
    - text signals (keywords)
    - optional metadata (e.g., account age, prior chargeback, IP mismatch)
    Returns:
      risk_level: low / medium / high
      risk_score: 0.0 – 1.0
      reasons   : list of triggers
    """

    TRIGGERS = [
        ("unauthorized", 0.30),
        ("not me", 0.25),
        ("i didn't order", 0.25),
        ("did not order", 0.25),
        ("scam", 0.30),
        ("fraud", 0.30),
        ("phishing", 0.30),
        ("hacked", 0.30),
        ("card charged", 0.30),
        ("unknown charge", 0.30),
        ("someone else", 0.20),
        ("suspicious", 0.20),
    ]

    def assess(self, message: str, meta: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        meta = meta or {}
        text = message.lower()

        risk_score = 0.05
        reasons: List[str] = []

        # text-based triggers
        for phrase, weight in self.TRIGGERS:
            if phrase in text:
                risk_score += weight
                reasons.append(f"Keyword '{phrase}' detected")

        # metadata-based heuristics
        if meta.get("has_prior_chargeback"):
            risk_score += 0.20
            reasons.append("Customer has prior chargeback history")

        if meta.get("account_age_days", 365) < 7:
            risk_score += 0.15
            reasons.append("Very new account (< 7 days)")

        if meta.get("high_order_value"):
            risk_score += 0.10
            reasons.append("High order value")

        if meta.get("multiple_failed_logins"):
            risk_score += 0.15
            reasons.append("Multiple failed login attempts")

        if meta.get("ip_country_mismatch"):
            risk_score += 0.15
            reasons.append("IP and payment country mismatch")

        # clamp between 0 and 1
        risk_score = max(0.0, min(1.0, risk_score))

        if risk_score < 0.30:
            level = "low"
        elif risk_score < 0.60:
            level = "medium"
        else:
            level = "high"

        return {
            "risk_level": level,
            "risk_score": round(risk_score, 2),
            "reasons": reasons or ["No strong fraud signals detected"],
        }


✅ Conclusion & Next Steps

In this notebook, we built a **5-agent orchestration system** for e-commerce support that can:

- Classify customer intent and urgency  
- Analyse emotional tone using rule-based sentiment  
- Estimate fraud risk with interpretable reasons  
- Generate helpful, context-aware replies  
- Decide whether to escalate the case and to which queue  
- Maintain a simple, lightweight conversation memory  


In [24]:
# ===========================
# Cell 6 — Reply Generator Agent
# ===========================

class ReplyAgent:
    """
    Generates natural replies using:
    - intent
    - urgency
    - sentiment label
    - fraud risk level
    """

    def create_reply(
        self,
        message: str,
        intent: str,
        urgency: str,
        sentiment: str,
        fraud_level: str,
    ) -> str:
        base_apology = "I'm sorry you're facing this issue. "
        friendly_opening = "Thanks for reaching out. "

        # core reply by intent
        if intent == "refund":
            reply = (
                base_apology
                + "I can help you with a refund for your order. "
                  "Please share your order ID and any supporting details."
            )

        elif intent == "cancellation":
            reply = (
                "I can help cancel your subscription or order. "
                "Please confirm the order ID and the reason for cancellation."
            )

        elif intent == "billing":
            reply = (
                base_apology
                + "I'll review the charges on your account. "
                  "Please send a screenshot of the invoice or the last 4 digits of the transaction ID."
            )

        elif intent == "account_issue":
            reply = (
                base_apology
                + "For your security, we'll help you regain access. "
                  "Please confirm your registered email so we can send reset instructions."
            )

        elif intent == "delivery_issue":
            reply = (
                base_apology
                + "I'll check the delivery status with our logistics team. "
                  "Please share your tracking ID or order number."
            )

        elif intent == "fraud_report":
            reply = (
                base_apology
                + "Security is our top priority. We'll immediately review this as a potential fraud case. "
                  "Please do not share any OTP with anyone and confirm the last 4 digits of the card used."
            )

        elif intent == "feedback":
            reply = friendly_opening + "We really appreciate your feedback. It helps us improve our service!"

        else:  # general_help and unknown
            reply = friendly_opening + "Could you please provide a few more details so I can assist you better?"

        # tone adjustment based on sentiment
        if sentiment in ("very_angry", "frustrated"):
            reply = "I completely understand how frustrating this must be. " + reply
        elif sentiment in ("positive", "very_positive"):
            reply = "Glad to hear from you! " + reply

        # mention fraud team for high risk
        if fraud_level == "high":
            reply += (
                " Our fraud investigation team will review this case on priority "
                "and may contact you for additional verification."
            )

        # urgency note
        if urgency in ("high", "critical"):
            reply += " We’ll treat this request as high priority."

        return reply


In [25]:
# ========================
# Cell 7 — Escalation Agent
# ========================

class EscalationAgent:
    """
    Decides whether the conversation must be escalated to a human agent.
    Considers:
    - intent urgency
    - sentiment label
    - fraud risk level
    """

    def check(
        self,
        intent_info: Dict[str, str],
        sentiment_info: Dict[str, Any],
        fraud_info: Dict[str, Any],
    ) -> Dict[str, Any]:
        reasons: List[str] = []
        escalate = False
        queue = "normal"

        # urgency-based escalation
        if intent_info["urgency"] in ("high", "critical"):
            escalate = True
            reasons.append(f"Urgency marked as {intent_info['urgency']}")

        # fraud-based escalation
        if fraud_info["risk_level"] == "high":
            escalate = True
            queue = "fraud_team"
            reasons.append("High fraud risk detected")

        # sentiment-based escalation
        if sentiment_info["label"] in ("very_angry", "frustrated"):
            escalate = True
            if queue == "normal":
                queue = "priority_support"
            reasons.append(f"Customer sentiment is {sentiment_info['label']}")

        if not reasons:
            reasons.append("No escalation required.")

        return {"escalate": escalate, "queue": queue, "reasons": reasons}


In [26]:
# ========================
# Cell 8 — Coordinator Agent
# ========================

class Coordinator:
    """
    Central Orchestrator for the multi-agent system.
    Steps:
      1. Store user message in memory
      2. Run IntentAgent, SentimentAgent, FraudAgent
      3. Generate reply via ReplyAgent
      4. Decide escalation via EscalationAgent
      5. Store assistant reply in memory
      6. Return structured JSON-style output
    """

    def __init__(self):
        self.intent_agent = IntentAgent()
        self.sentiment_agent = SentimentAgent()
        self.fraud_agent = FraudAgent()
        self.reply_agent = ReplyAgent()
        self.escalation_agent = EscalationAgent()
        self.memory = Memory()

    def ask(self, message: str, meta: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        meta = meta or {}

        # 1) memory update with user message
        self.memory.add("user", message, meta)

        # 2) run agents
        intent_info = self.intent_agent.classify(message)
        sentiment_info = self.sentiment_agent.analyze(message)
        fraud_info = self.fraud_agent.assess(message, meta)

        # 3) agent reply
        reply_text = self.reply_agent.create_reply(
            message=message,
            intent=intent_info["intent"],
            urgency=intent_info["urgency"],
            sentiment=sentiment_info["label"],
            fraud_level=fraud_info["risk_level"],
        )

        # 4) escalation
        escalation_info = self.escalation_agent.check(
            intent_info=intent_info,
            sentiment_info=sentiment_info,
            fraud_info=fraud_info,
        )

        # 5) save assistant reply in memory
        self.memory.add(
            "assistant",
            reply_text,
            {
                "intent": intent_info,
                "sentiment": sentiment_info,
                "fraud": fraud_info,
                "escalation": escalation_info,
            },
        )

        # 6) final combined output
        output = {
            "input_message": message,
            "meta": meta,
            "intent": intent_info,
            "sentiment": sentiment_info,
            "fraud_assessment": fraud_info,
            "reply": reply_text,
            "escalation": escalation_info,
            "recent_memory": self.memory.to_list(6),
            "short_context": self.memory.get_recent_context(4),
        }

        return output


In [27]:
# ===================================
# Cell 9 — Test the Multi-Agent System
# ===================================

agent = Coordinator()

# Example test cases (tu apne examples bhi add kar sakta hai)
test_cases = [
    {
        "text": "I see an unknown charge on my card and it's not me. This looks like fraud!",
        "meta": {
            "has_prior_chargeback": False,
            "account_age_days": 3,
            "high_order_value": True,
            "ip_country_mismatch": True,
            "multiple_failed_logins": True,
        },
    },
    {
        "text": "My parcel has still not been delivered and I'm very upset with this terrible service.",
        "meta": {
            "account_age_days": 120,
            "high_order_value": False,
        },
    },
    {
        "text": "Hey, I just want to cancel my subscription. Thanks!",
        "meta": {
            "account_age_days": 400,
        },
    },
    {
        "text": "Why was I charged twice on my invoice? This is unacceptable.",
        "meta": {
            "account_age_days": 60,
            "high_order_value": True,
        },
    },
    {
        "text": "Loved the quick delivery, you guys are awesome!",
        "meta": {
            "account_age_days": 200,
        },
    },
]

for case in test_cases:
    msg = case["text"]
    meta = case.get("meta", {})
    print("USER:", msg)
    out = agent.ask(msg, meta)
    print(json.dumps(out, indent=2))
    print("-" * 80)


USER: I see an unknown charge on my card and it's not me. This looks like fraud!
{
  "input_message": "I see an unknown charge on my card and it's not me. This looks like fraud!",
  "meta": {
    "has_prior_chargeback": false,
    "account_age_days": 3,
    "high_order_value": true,
    "ip_country_mismatch": true,
    "multiple_failed_logins": true
  },
  "intent": {
    "intent": "fraud_report",
    "urgency": "critical"
  },
  "sentiment": {
    "label": "frustrated",
    "score": -1,
    "positive_matches": 0,
    "negative_matches": 1
  },
  "fraud_assessment": {
    "risk_level": "high",
    "risk_score": 1.0,
    "reasons": [
      "Keyword 'not me' detected",
      "Keyword 'fraud' detected",
      "Keyword 'unknown charge' detected",
      "Very new account (< 7 days)",
      "High order value",
      "Multiple failed login attempts",
      "IP and payment country mismatch"
    ]
  },
  "reply": "I completely understand how frustrating this must be. I'm sorry you're facing thi